## The FDA pharmacogenomics biomarkers source table
* Obtained from https://www.fda.gov/drugs/science-and-research-drugs/table-pharmacogenomic-biomarkers-drug-labeling
* date: 06/25/2025

In [23]:
## Load necessary packages
import os
import pandas as pd
import glob
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Define the version number
version_number = "07_01_2025"
deployment_date = "2025-07-01"

## Load files and convert them into separate node & edge files
* check all imported file structure

In [24]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/FDA_Pharmacogenomic_biomarkers_in_Drug_labeling/'

## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/FDA_pharmacogenomics_biomarkers_parsed_node_{version_number}.tsv'
download_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/FDA_pharmacogenomics_biomarkers_parsed_edge_{version_number}.tsv'

In [25]:
## Check all csv files being read
for f in os.listdir(raw_files_path):
    if f.endswith('.csv'):
        print(f)

Table_of_Pharmacogenomic_Biomarkers_in_Drug_Labeling_FDA.csv


In [26]:
## Read each individual csv files
FDA_biomarkers_df = pd.read_csv(raw_files_path + 'Table_of_Pharmacogenomic_Biomarkers_in_Drug_Labeling_FDA.csv')

FDA_biomarkers_df.head(5)

,Drug,Therapeutic Area*,Biomarker†,Labeling Sections
0,Articaine and Epinephrine (1),Anesthesiology,G6PD,Warnings and Precautions
1,Articaine and Epinephrine (2),Anesthesiology,Nonspecific (Congenital Methemoglobinemia),Warnings and Precautions
2,Bupivacaine (1),Anesthesiology,G6PD,Warnings
3,Bupivacaine (2),Anesthesiology,Nonspecific (Congenital Methemoglobinemia),Warnings
4,Chloroprocaine (1),Anesthesiology,G6PD,Warnings


In [34]:
print(len(FDA_biomarkers_df))

608


## Notice: Multiple types of labeling sections
* full list here:
* 

In [27]:
## check unique labeling sections
unique_labeling_values = FDA_biomarkers_df['Labeling Sections'].unique()
print("All possible labeling are here: " ,unique_labeling_values)
print("Full length of the list: ", len(unique_labeling_values))

All possible labeling are here:  ['Warnings and Precautions' 'Warnings'
 'Boxed Warning, Warnings and Precautions, Use in Specific Populations, Patient Counseling Information'
 'Contraindications, Warnings and Precautions, Clinical Pharmacology'
 'Contraindications, Warnings, Clinical Pharmacology'
 'Warnings and Precautions, Clinical Pharmacology'
 'Use in Specific Populations'
 'Use in Specific Populations, Clinical Pharmacology'
 'Warnings, Precautions, Clinical Pharmacology'
 'Warnings and Precautions, Drug Interactions, Use in Specific Populations, Clinical Pharmacology'
 'Warnings, Precautions'
 'Boxed Warning, Contraindications, Warnings, Precautions, Adverse Reactions, Clinical Pharmacology'
 'Boxed Warning, Warnings and Precautions, Use in Specific Populations, Clinical Pharmacology, Patient Counseling Information'
 'Drug Interactions, Clinical Pharmacology'
 'Boxed Warning, Warnings and Precautions, Clinical Pharmacology'
 'Clinical Pharmacology' 'Overdosage'
 'Dosage and Adm

## Helper function to obtain all unique labels from the full list
* Information based on Figure A: Highlights of perscribing information,
* Sections on figure:
    * Warning: Title of warning
    * Recent major changes
    * Indications and usage
    * Dosage and administration
    * Dosage forms and strengths
    * Contraindications
    * Warnings and precautions
    * Adverse reactions
    * Drug interactions
    * Use in specific populations
* https://www.fda.gov/drugs/fdas-labeling-resources-human-prescription-drugs/frequently-asked-questions-about-labeling-prescription-medicines?utm_source=chatgpt.com
* 

In [28]:
def find_unique_words(string_list):
    unique_words = set()
    for s in string_list:
        words = s.split(',')
        for word in words:
            cleaned_word = word.strip()
            if cleaned_word:
                unique_words.add(cleaned_word)
    return unique_words

In [29]:
unique_labels = find_unique_words(unique_labeling_values)
print("All possible unique labels: ", unique_labels)
print(len(unique_labels))

All possible unique labels:  {'Use in Specific Populations', 'Warnings', 'Indications and Usage', 'Dosage and Administrations', 'Adverse Reactions', 'Warnings and Precautions', 'Adverse Interactions', 'Dosage and Administration', 'Warning and Precautions', 'Indication and Usage', 'Patient Counseling Information', 'Boxed Warning', 'Precautions', 'Drug Interactions', 'Contraindications', 'Overdosage', 'Clinical Pharmacology', 'Clinical Studies'}
18


In [30]:
import pandas as pd

## Define the data table
## Information based on Figure A: Highlights of perscribing information,
data = {
    "Label": [
        "Boxed Warning",
        "Contraindications",
        "Warnings",
        "Warnings and Precautions", ## Group 'Warning and Precautions' here
        "Precautions",
        "Overdosage",
        "Adverse Reactions",
        "Adverse Interactions",
        "Drug Interactions",
        "Use in Specific Populations",
        "Patient Counseling Info",
        "Indications and Usage", ## Group 'Indication and Usage' here
        "Dosage and Administrations", ## Group 'Dosage and Administration' here
        "Clinical Pharmacology",
        "Clinical Studies"
    ],
    "Type": [
        "Safety (critical)",
        "Safety (critical)",
        "Safety (critical)",
        "Safety",
        "Safety",
        "Safety",
        "Safety",
        "Safety",
        "Safety",
        "Guidance",
        "Communication",
        "Usage Guidance",
        "Usage Guidance",
        "Informational",
        "Informational"
    ],
    "Severity/Priority Level": [
        "Highest",
        "High",
        "High",
        "Medium-High",
        "Medium-High",
        "Medium-High",
        "Medium",
        "Medium",
        "Medium",
        "Medium",
        "Medium",
        "Low",
        "Low",
        "Low",
        "Low"
    ]
}

# Create the DataFrame
labeling_type_and_severity_df = pd.DataFrame(data)

# Display the DataFrame
print(labeling_type_and_severity_df)

                          Label               Type Severity/Priority Level
0                 Boxed Warning  Safety (critical)                 Highest
1             Contraindications  Safety (critical)                    High
2                      Warnings  Safety (critical)                    High
3      Warnings and Precautions             Safety             Medium-High
4                   Precautions             Safety             Medium-High
5                    Overdosage             Safety             Medium-High
6             Adverse Reactions             Safety                  Medium
7          Adverse Interactions             Safety                  Medium
8             Drug Interactions             Safety                  Medium
9   Use in Specific Populations           Guidance                  Medium
10      Patient Counseling Info      Communication                  Medium
11        Indications and Usage     Usage Guidance                     Low
12   Dosage and Administr

## Now some examples to discuss and decide the priority of which biomarkers subtype to assign
* example 1: multiple high severity classes
* 'Boxed Warning, Contraindications, Warnings and Precautions, Adverse Reactions, Patient Counseling Information'
* example 2: a mix of high, medium, and low classes
* 'Warnings and Precautions, Adverse Reactions, Patient Counseling Information'

In [32]:
# Filter rows
filtered_df = FDA_biomarkers_df[FDA_biomarkers_df['Labeling Sections'] 
    == "Boxed Warning, Contraindications, Warnings and Precautions, Adverse Reactions, Patient Counseling Information"]

print(filtered_df)

            Drug Therapeutic Area* Biomarker†  \
595  Pegloticase      Rheumatology       G6PD   

                                     Labeling Sections  
595  Boxed Warning, Contraindications, Warnings and...  


In [33]:
# Filter rows
filtered_df = FDA_biomarkers_df[FDA_biomarkers_df['Labeling Sections'] 
    == "Warnings and Precautions, Adverse Reactions, Patient Counseling Information"]

print(filtered_df)

               Drug Therapeutic Area*  \
49      Dapsone (2)       Dermatology   
319  Dabrafenib (2)          Oncology   

                                     Biomarker†  \
49   Nonspecific (Congenital Methemoglobinemia)   
319                                        G6PD   

                                     Labeling Sections  
49   Warnings and Precautions, Adverse Reactions, P...  
319  Warnings and Precautions, Adverse Reactions, P...  


In [ ]:
## Drugbank result
## https://go.drugbank.com/pharmaco/genomics/DBSNPE001209?utm_source=chatgpt.com

## G6PD deficiency -> Increased risk of hemolytic anemia.